In [12]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [17]:
import dotenv
%load_ext dotenv
%dotenv

In [13]:
import nest_asyncio
nest_asyncio.apply()

In [14]:
from llama_index.core import SimpleDirectoryReader

# Load documents
documents = SimpleDirectoryReader(input_files=["./datasets/lora.pdf"]).load_data()

In [3]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes= splitter.get_nodes_from_documents(documents)

In [4]:
from llama_index.core import VectorStoreIndex
vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [5]:
from  llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [{"key": "page_label", "value": "2"}]
    )
)

response = query_engine.query(
    " what are Transformer Language Models?"
)

In [6]:
print(str(response))

Transformer Language Models are autoregressive models used for tasks like language modeling. They are based on the Transformer architecture and are capable of generating text by predicting the next word in a sequence based on the context provided. These models are pre-trained on large datasets and can be fine-tuned for specific downstream tasks such as summarization, machine reading comprehension, and natural language to SQL conversion.


In [7]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'lora.pdf', 'file_path': 'datasets\\lora.pdf', 'file_type': 'application/pdf', 'file_size': 1609513, 'creation_date': '2024-06-18', 'last_modified_date': '2024-06-17'}


In [9]:
from typing import List
from llama_index.core.vector_stores import FilterCondition
from llama_index.core.tools import FunctionTool

def vector_query_engine(
        query: str,
        page_numbers: List[int]
    ) -> str:
        """ Perform a vector search over an index.
        query (str): the string query to be embedded.
        page_numbers (List[int]): Filter by set of pages. Leave blank id we want to perform a vector  search 
        over all pages. otherwise, filter by the set of specified pages
        
        """
        
        metadata_dicts = [
            {"key": "page_label", "value": p} for p in page_numbers
        ]
        query_engine = vector_index.as_query_engine(
            similarity_top_k=2,
            filters=MetadataFilters.from_dicts(
                metadata_dicts,
                condition=FilterCondition.OR
            ),
        )
        response = query_engine.query(query)
        return response
    
vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query_engine
)

In [22]:
import openai

from llama_index.llms import LlamaIndex

# Set your OpenAI API key
# Initialize the LlamaIndex with OpenAI
llm = LlamaIndex(api_key=openai.api_key, model="gpt-3.5-turbo", temperature=0)

response = llm.predict_and_call(
    [vector_query_tool],
    input="what are Subspace similarity between different random seeds as described in page 11?",
    stop=["\n\n"],
    verbose=True
)

print(response)


ImportError: cannot import name 'LlamaIndex' from 'llama_index.llms' (unknown location)

In [20]:
import openai

llm = openai.OpenAI(model="gpt-3.5-turbo", temperature=0)
response = llm.predict_and_call(
  [vector_query_tool],
  input="what are Subspace similarity between different random seeds as described in page 11?",
  stop=["\n\n"],
  verbose=True
)

TypeError: OpenAI.__init__() got an unexpected keyword argument 'model'

In [ ]:
response = query_engine.query("what are Subspace similarity between different random seeds as described in page 11?")

